In [1]:
from math import atan2
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource
output_notebook()
#https://nbviewer.jupyter.org/github/bokeh/bokeh-notebooks/tree/master/tutorial/

Loading BokehJS ...

In [2]:
file_path = "../data/out-3.txt"

'''
time_stamp
px_state
py_state
v_state
yaw_angle_state
yaw_rate_state
sensor_type
NIS
px_measured
py_measured
px_ground_truth
py_ground_truth
vx_ground_truth
vy_ground_truth
'''

all_px_truth = []
all_py_truth = []
all_px_radar = []
all_py_radar = []
all_px_lidar = []
all_py_lidar = []
all_px_state = []
all_py_state = []
all_nis_lidar = []
all_nis_radar = []
all_yaw_truth = []
all_yaw_state = []


with open(file_path) as f:
  count = 0
    
  for line in f:
    if count == 0:
      count = 1
      continue
    
    data = line.split()
    timestamp = int(data[0])
    px_state = float(data[1])
    py_state = float(data[2])
    v_state = float(data[3])
    yaw_state = float(data[4])
    yawrate_state = float(data[5])
    sensor_type = data[6]
    nis = float(data[7])
    px_measured = float(data[8])
    py_measured = float(data[9])
    px_truth = float(data[10])
    py_truth = float(data[11])
    vx_truth = float(data[12])
    vy_truth = float(data[13])
    yaw_truth = atan2(vy_truth, vx_truth)

    all_px_truth.append(px_truth)
    all_py_truth.append(py_truth)
    all_px_state.append(px_state)
    all_py_state.append(py_state)
    all_yaw_truth.append(yaw_truth)
    all_yaw_state.append(yaw_state)
    
    if sensor_type == "lidar":
      all_nis_lidar.append(nis)
      all_px_lidar.append(px_measured)
      all_py_lidar.append(py_measured)
    
    if sensor_type == "radar":
      all_nis_radar.append(nis)
      all_px_radar.append(px_measured)
      all_py_radar.append(py_measured)

In [3]:
radar_source = ColumnDataSource(data = {
    'x' : all_px_radar,
    'y' : all_py_radar 
})

lidar_source = ColumnDataSource(data = {
    'x' : all_px_lidar,
    'y' : all_py_lidar 
})

state_source = ColumnDataSource(data = {
    'x' : all_px_state,
    'y' : all_py_state, 
    'angle' : all_yaw_state
})

truth_source = ColumnDataSource(data = {
    'x' : all_px_truth,
    'y' : all_py_truth, 
    'angle' : all_yaw_truth
})

In [4]:
p = figure(plot_width = 1000, plot_height = 700)

p.square(
  'x', 'y', size = 5, 
  fill_color = "orange",
  line_color = "orange",
  fill_alpha = 1,
  line_width = 1, 
  legend = "radar measurements", 
  source = radar_source)

p.circle(
  'x', 'y', size = 15, 
  fill_color = "green",
  line_color = "white",
  fill_alpha = 0.2,
  line_width = 1, 
  legend = "lidar measurements", 
  source = lidar_source)

p.triangle(
  'x', 'y', size = 7, 
  fill_color = "violet",
  line_color = "violet",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 0.5, 
  legend = "ground truth",
  source = truth_source)


p.triangle(
  'x', 'y', size = 2, 
  fill_color = "black",
  line_color = "black",
  fill_alpha = 1,
  angle = 'angle', 
  line_width = 1, 
  legend = "state predictions", 
  source = state_source)

p.legend.location = "bottom_right"
show(p)

In [5]:
p2 = figure(plot_width = 1000, plot_height = 700)

p2.square(
  'x', 'y', size = 5, 
  fill_color = "violet",
  line_color = "violet",
  fill_alpha = 1,
  line_width = 1, 
  legend = "radar measurements", 
  source = radar_source)

p2.circle(
  'x', 'y', size = 10, 
  fill_color = "green",
  line_color = "white",
  fill_alpha = 0.4,
  line_width = 1, 
  legend = "lidar measurements", 
  source = lidar_source)

p2.line(all_px_state, all_py_state, line_width = 2, color='orange', legend = "state predictions")
p2.line(all_px_truth, all_py_truth, line_dash = "4 4", line_width = 1, color='navy', legend = "ground truth")

p2.legend.location = "bottom_right"
show(p2)

In [6]:
nis_length = len(all_nis_radar)
x_radar = [i for i in range(nis_length)]

p3 = figure(plot_width = 700, plot_height = 700)

p3.circle(x_radar, all_nis_radar, size = 5, fill_color="orange", line_color = "orange", legend = "radar_nis")
p3.line([0, nis_length], [7.815, 7.815], line_width = 2, color='violet', legend = "chi squared 0.95 - 3DOF")

p3.legend.location = "top_right"

show(p3)

In [7]:
nis_length = len(all_nis_lidar)
x_lidar = [i for i in range(nis_length)]

p4 = figure(plot_width = 700, plot_height = 700)

p4.circle(x_lidar, all_nis_lidar, size = 5, fill_color="violet", line_color = "violet", legend = "lidar_nis")
p4.line([0, nis_length], [5.991, 5.991], line_width = 2, color='orange', legend = "chi squared 0.95 - 2DOF")

p4.legend.location = "top_right"

show(p4)